In [41]:
from pathlib import Path
import os
import pandas as pd
Path.cont = lambda x: list(os.scandir(x)) # scandir gives all content of a dir 
filedir = 'RIVM_data'
files = Path.cont(filedir)

def generate_df(filedir,filename,i):
    if len(filename.name) > 30:
        df = pd.read_csv(filedir+'//'+filename.name,encoding = 'cp1252',delimiter=';')
    else:
        df = pd.read_csv(filedir+'//'+filename.name,encoding = 'cp1252',skiprows=1)

    if 'BevAant' in df.columns:
        df['BevAant'] = pd.to_numeric(df['BevAant'])
    df['Aantal'] = pd.to_numeric(df['Aantal'])
    df.rename(columns={'Aantal per 100.000 inwoners':'Aantal per 100000 inwoners'},inplace=True)
    df['Datum'] = filename.name[:10]
    df['Count'] = str(i)
    print(filename.name)
    # Gemnr,Gemeente,Aantal,BevAant,Aantal per 100.000 inwoners 
    # Gemeente,Gemnr,Aantal
    return df
for i,filename in enumerate(files):
    print(filename.name)
    print(generate_df(filedir,filename,i).columns)
df = pd.concat([generate_df(filedir,filename,i) for i,filename in enumerate(files)])
num_csv = len(files)
df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_aantal.head()

03-03-2020 20200308095848klik_corona03032020.csv
03-03-2020 20200308095848klik_corona03032020.csv
Index(['id', 'Gemeente', 'Indicator', 'Aantal', 'Datum', 'Count'], dtype='object')
03-04-2020 20200308095940isthisokklik_corona04032020.csv
03-04-2020 20200308095940isthisokklik_corona04032020.csv
Index(['id', 'Gemeente', 'Indicator', 'Aantal', 'Datum', 'Count'], dtype='object')
03-05-2020 20200308100010klik_corona05032020.csv
03-05-2020 20200308100010klik_corona05032020.csv
Index(['id', 'Gemeente', 'Indicator', 'Aantal', 'Datum', 'Count'], dtype='object')
03-06-2020 20200308100025klik_corona06032020_rec_0.csv
03-06-2020 20200308100025klik_corona06032020_rec_0.csv
Index(['id', 'Gemeente', 'Indicator', 'Aantal', 'Datum', 'Count'], dtype='object')
03-07-2020 20200308031948klik_corona07032020.csv
03-07-2020 20200308031948klik_corona07032020.csv
Index(['id', 'Gemeente', 'Indicator', 'Aantal', 'Datum', 'Count'], dtype='object')
03-08-2020 20200308211244klik_corona08032020_rectificatie.csv
03-08

KeyError: 'Aantal per 100000 inwoners'

In [36]:
df_aantal = df.pivot(index='Gemeente', columns='Count', values='Aantal').reset_index()
df_relative = df.pivot(index='Gemeente', columns='Count', values='Aantal per 100000 inwoners').reset_index()
df_aantal.head()

ValueError: Index contains duplicate entries, cannot reshape

In [23]:
# df.columns = ['_'.join(col) for col in df.columns.values]
# df

In [24]:
import altair as alt
from vega_datasets import data

url1 = 'https://cartomap.github.io/nl/rd/gemeente_2020.topojson'
url2 = 'https://cartomap.github.io/nl/wgs84/gemeente_2020.topojson' 

gemeentes = alt.topo_feature(url2, 'gemeente_2020')

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import display
w = widgets.IntSlider()

columns = [str(day) for day in range(0, 30)]
slider = alt.binding_range(min=0, max=30, step=1)
select_day = alt.selection_single(name="day", fields=['day'],
                                   bind=slider, init={'day': 1})

def generate_chart(col):
  # print(col)

  chart = alt.Chart(gemeentes).mark_geoshape().properties(
    width=800,
    height=500
    ).transform_lookup(
    lookup='properties.statnaam',
    from_=alt.LookupData(df, 'Gemeente_', [col]),
    default= '0'
    ).encode(
    color=alt.Color(col+':Q',scale=alt.Scale(type='symlog')),
    tooltip=['properties.statnaam:N', col+':Q']
    )
  return chart

generate_chart('Aantal_1')|generate_chart('Aantal_9')


alt.HConcatChart(...)

In [25]:
import altair as alt
import pandas as pd
from vega_datasets import data
columns = [str(day) for day in range(0, num_csv)]

slider = alt.binding_range(min=0, max=num_csv-1, step=1)
select_year = alt.selection_single(name="day", fields=['day'],
                                   bind=slider, init={'day': num_csv-1})
def serve_chart(df, legend):
    chart = alt.Chart(gemeentes).mark_geoshape(
        stroke='black',
        strokeWidth=0.05
    ).transform_lookup(
        lookup='properties.statnaam',
        from_=alt.LookupData(df, 'Gemeente', columns)
    ).transform_fold(
        columns, as_=['day', legend]
    ).transform_calculate(
        day='parseInt(datum.day)',
        legend='isValid(datum.'+legend+') ? datum.'+legend+' : -1'  
    ).encode(
        color = alt.condition(
            'datum.'+legend+' > 0',
            alt.Color(legend+':Q', scale=alt.Scale(scheme='redyellowgreen', type='symlog',domain=[200, 0])),
            alt.value('#dbe9f6')
        )).add_selection(
        select_year
    ).properties(
        width=350,
        height=400
    ).transform_filter(
        select_year
    )
    
    return chart
serve_chart(df_aantal, 'aantal') | serve_chart(df_relative, 'aantal')

alt.HConcatChart(...)

In [26]:
alt.hconcat(
    serve_chart(df_aantal, 'absoluut'), serve_chart(df_relative, 'relatief')
).resolve_scale(
    color='independent'
).configure_view(
    stroke=None
)

alt.HConcatChart(...)

In [65]:
df_aantal[df_aantal['Gemeente']=='Uden']

Count,Gemeente,0,1,2,3
296,Uden,53.0,60.0,60.0,71.0


In [66]:
df_relative[df_relative['Gemeente']=='Uden']

Count,Gemeente,0,1,2,3
296,Uden,126.8,143.6,143.6,169.9


In [37]:
df['Gemeente'].unique()

array(['Aa en Hunze', 'Aalsmeer', 'Aalten', 'Achtkarspelen',
       'Alblasserdam', 'Albrandswaard', 'Alkmaar', 'Almelo', 'Almere',
       'Alphen aan den Rijn', 'Alphen-Chaam', 'Altena', 'Ameland',
       'Amersfoort', 'Amstelveen', 'Amsterdam', 'Apeldoorn', 'Appingedam',
       'Arnhem', 'Assen', 'Asten', 'Baarle-Nassau', 'Baarn',
       'Barendrecht', 'Barneveld', 'Beek', 'Beekdaelen', 'Beemster',
       'Beesel', 'Berg en Dal', 'Bergeijk', 'Bergen (L)', 'Bergen (NH)',
       'Bergen op Zoom', 'Berkelland', 'Bernheze', 'Best', 'Beuningen',
       'Beverwijk', 'Bladel', 'Blaricum', 'Bloemendaal',
       'Bodegraven-Reeuwijk', 'Boekel', 'Borger-Odoorn', 'Borne',
       'Borsele', 'Boxmeer', 'Boxtel', 'Breda', 'Brielle', 'Bronckhorst',
       'Brummen', 'Brunssum', 'Bunnik', 'Bunschoten', 'Buren',
       'Capelle aan den IJssel', 'Castricum', 'Coevorden', 'Cranendonck',
       'Cuijk', 'Culemborg', 'Dalfsen', 'Dantumadiel', 'De Bilt',
       'De Fryske Marren', 'De Ronde Venen', 'De Wo